In [1]:
#!pip install bertopic datasets accelerate bitsandbytes xformers adjustText

# imports
import pandas as pd
import ipywidgets as widgets
import math
import os
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
from octis.dataset.dataset import Dataset
from octis.optimization.optimizer import Optimizer
from skopt.space.space import Real, Categorical, Integer
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
import time
import en_core_web_sm


optimizer=Optimizer()
nlp = en_core_web_sm.load()

/sw/arch/RHEL8/EB_production/2022/software/Python/3.10.4-GCCcore-11.3.0/lib/python3.10/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [33]:
from sentence_transformers import SentenceTransformer, models
import torch
import os

# Load your pre-trained model
model_name = 'vinai/bertweet-base'
word_embedding_model = models.Transformer(model_name)

# Define the custom pooling layer with mean and max pooling combined
class CustomPoolingLayer(models.Pooling):
    def __init__(self, word_embedding_dimension):
        super(CustomPoolingLayer, self).__init__(word_embedding_dimension)
    
    def mean_pooling(self, token_embeddings, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask
    
    def max_pooling(self, token_embeddings, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        token_embeddings[input_mask_expanded == 0] = -1e9  # Set padding tokens to large negative value
        max_embeddings = torch.max(token_embeddings, 1)[0]
        return max_embeddings

    def forward(self, features):
        token_embeddings = features['token_embeddings']
        attention_mask = features['attention_mask']
        
        # Mean pooling
        mean_pooled = self.mean_pooling(token_embeddings, attention_mask)
        
        # Max pooling
        max_pooled = self.max_pooling(token_embeddings, attention_mask)
        
        # Concatenate mean and max pooled embeddings
        sentence_embedding = torch.cat((mean_pooled, max_pooled), 1)
        
        features.update({'sentence_embedding': sentence_embedding})
        return features

# Get the embedding dimension
embedding_dim = word_embedding_model.get_word_embedding_dimension()

# Create the custom pooling layer
custom_pooling = CustomPoolingLayer(embedding_dim)

# Define the SentenceTransformer model with the custom pooling
sentence_transformer_model = SentenceTransformer(modules=[word_embedding_model, custom_pooling])

In [34]:
#SETUP
DATA = "trump"
EM = "vinai/bertweet-base"
EM2 = "all-mpnet-base-v1"
EM3 = sentence_transformer_model

In [2]:
from data_trump import DataLoader

[nltk_data] Downloading package punkt to
[nltk_data]     /home/pspaargaren/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from sentence_transformers import SentenceTransformer
# Prepare data
dataset, custom = "trump", True
data_loader = DataLoader(dataset)
_, timestamps = data_loader.load_docs()
data = data_loader.load_octis(custom)
data = [" ".join(words) for words in data.get_corpus()]

# Extract embeddings
model = SentenceTransformer("all-mpnet-base-v2")
embeddings = model.encode(data, show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/591 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1383 [00:00<?, ?it/s]

In [37]:
from sentence_transformers import SentenceTransformer
dataset = Dataset()
dataset.load_custom_dataset_from_folder(DATA)
docs = [" ".join(words) for words in dataset.get_corpus()]
embedding_model = EM3#SentenceTransformer(EM2) #EM3
embeddings = embedding_model.encode(docs, show_progress_bar=True)


Batches:   0%|          | 0/1383 [00:00<?, ?it/s]

In [40]:
dataset, custom = DATA, True

In [4]:
from evaluation_trump import Trainer

2024-06-11 19:39:32.971971: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-11 19:39:33.406317: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pspaargaren/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# BERTopic

In [6]:
#TC & TD CALCULATION
from bertopic import BERTopic
from bertopic.representation import MaximalMarginalRelevance

representation_model = MaximalMarginalRelevance(diversity=None)
for i in range(3):
    custom = True
    params = {
        "embedding_model": "all-mpnet-base-v2",
        "nr_topics": [(i+1)*10 for i in range(5)],
        "min_topic_size": 15,
        #"diversity": None,
        "verbose": True
    }

    trainer = Trainer(dataset=dataset,
                        model_name="BERTopic",
                        params=params,
                        bt_embeddings=embeddings,
                        custom_dataset=custom,
                        representation_model = representation_model,
                        verbose=True)
    results = trainer.train(save=f"results/trump/scores/TR_bertopic_{i+1}")

2024-06-11 19:40:19,326 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-11 19:40:43,956 - BERTopic - Dimensionality - Completed ✓
2024-06-11 19:40:43,959 - BERTopic - Cluster - Start clustering the reduced embeddings
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been us

Results
npmi: -0.001786529033318741
diversity: 0.6777777777777778
 


2024-06-11 19:40:57,033 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-11 19:41:20,142 - BERTopic - Dimensionality - Completed ✓
2024-06-11 19:41:20,145 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-11 19:41:22,149 - BERTopic - Cluster - Completed ✓
2024-06-11 19:41:22,150 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-11 19:41:23,319 - BERTopic - Representation - Completed ✓
2024-06-11 19:41:23,320 - BERTopic - Topic reduction - Reducing number of topics
2024-06-11 19:41:24,195 - BERTopic - Topic reduction - Reduced number of topics from 350 to 20


Results
npmi: -0.02716461578672935
diversity: 0.7105263157894737
 


2024-06-11 19:41:32,556 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-11 19:41:57,249 - BERTopic - Dimensionality - Completed ✓
2024-06-11 19:41:57,251 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-11 19:41:59,279 - BERTopic - Cluster - Completed ✓
2024-06-11 19:41:59,280 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-11 19:42:00,466 - BERTopic - Representation - Completed ✓
2024-06-11 19:42:00,467 - BERTopic - Topic reduction - Reducing number of topics
2024-06-11 19:42:01,364 - BERTopic - Topic reduction - Reduced number of topics from 347 to 30


...

Results
npmi: -0.04894062064103474
diversity: 0.7473684210526316
 


2024-06-11 19:48:11,334 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-11 19:48:34,469 - BERTopic - Dimensionality - Completed ✓
2024-06-11 19:48:34,471 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-11 19:48:36,462 - BERTopic - Cluster - Completed ✓
2024-06-11 19:48:36,462 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-11 19:48:37,620 - BERTopic - Representation - Completed ✓
2024-06-11 19:48:37,621 - BERTopic - Topic reduction - Reducing number of topics
2024-06-11 19:48:38,513 - BERTopic - Topic reduction - Reduced number of topics from 349 to 30


Results
npmi: -0.021781731775790816
diversity: 0.7517241379310344
 


2024-06-11 19:48:47,382 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


SystemError: CPUDispatcher(<function nn_descent at 0x15466ee28ca0>) returned a result with an exception set

In [13]:
#TOPIC EXTRACTION
custom = True
params = {
    "embedding_model": embedding_model,
    "nr_topics": 15,
    "min_topic_size": 10,
    #"diversity": None,
    "verbose": True
}

trainer = Trainer(dataset=dataset,
                    model_name="BERTopic",
                    params=params,
                    bt_embeddings=embeddings,
                    custom_dataset=custom,
                    verbose=True)
results = trainer.train(save=f"results/trump/topics/TR_bertopic")

2024-06-11 11:19:42,805 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-11 11:20:06,796 - BERTopic - Dimensionality - Completed ✓
2024-06-11 11:20:06,799 - BERTopic - Cluster - Start clustering the reduced embeddings
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been us

Results
npmi: -0.04377524861085428
diversity: 0.8071428571428572
 


# LDA

In [24]:
for i, random_state in enumerate([0, 21, 42]):
    dataset, custom = DATA, True
    params = {"num_topics": [(i+1)*10 for i in range(5)], "random_state": random_state}

    trainer = Trainer(dataset=dataset,
                      model_name="LDA",
                      params=params,
                      custom_dataset=custom,
                      verbose=True)
    results = trainer.train(save=f"results/trump/scores/TR_lda_{i+1}")

Results
npmi: -0.00691144547486858
diversity: 0.48
 
Results
npmi: -0.005003465267478509
diversity: 0.425
 
Results
npmi: -0.004333717218804874
diversity: 0.5133333333333333
 
Results
npmi: -0.013484627735113932
diversity: 0.5475
 
Results
npmi: -0.02873932987802867
diversity: 0.556
 
Results
npmi: -0.008732834718608936
diversity: 0.41
 
Results
npmi: -0.0032195570281142545
diversity: 0.44
 
Results
npmi: -0.011184204528511061
diversity: 0.52
 
Results
npmi: -0.010834766671676585
diversity: 0.5275
 
Results
npmi: -0.020161391882537752
diversity: 0.596
 
Results
npmi: -0.004250980433560531
diversity: 0.45
 
Results
npmi: -0.006889694250977099
diversity: 0.47
 
Results
npmi: -0.005643503573800874
diversity: 0.48333333333333334
 
Results
npmi: -0.01100933809236911
diversity: 0.55
 
Results
npmi: -0.018123237177004415
diversity: 0.574
 


In [15]:
#TOPIC CREATION:
dataset, custom = DATA, True
params = {"num_topics": 15, "random_state": 42}#[(i+1)*10 for i in range(5)], "random_state": random_state}

trainer = Trainer(dataset=dataset,
                  model_name="LDA",
                  params=params,
                  custom_dataset=custom,
                  verbose=True)
results = trainer.train(save=f"results/trump/topics/TR_lda")

Results
npmi: -0.005812970405126746
diversity: 0.3933333333333333
 


# NMF

In [9]:
for i, random_state in enumerate([0, 21, 42]):
    dataset, custom = DATA, True
    params = {"num_topics": [(i+1)*10 for i in range(5)], "random_state": random_state}

    trainer = Trainer(dataset=dataset,
                      model_name="NMF",
                      params=params,
                      custom_dataset=custom,
                      verbose=True)
    results = trainer.train(save=f"results/trump/scores/TR_nmf_{i+1}")

Results
npmi: -0.005742839116935675
diversity: 0.4
 
Results
npmi: 0.007421120689733075
diversity: 0.42
 
Results
npmi: 0.014766098200964957
diversity: 0.3933333333333333
 
Results
npmi: 0.016208854270394913
diversity: 0.3375
 
Results
npmi: 0.01722279340526775
diversity: 0.348
 
Results
npmi: -0.00391705970481047
diversity: 0.46
 
Results
npmi: 0.007975363716605935
diversity: 0.415
 
Results
npmi: 0.02100240700031534
diversity: 0.38
 
Results
npmi: 0.010374395636827634
diversity: 0.36
 
Results
npmi: 0.00693951977530063
diversity: 0.316
 
Results
npmi: -0.00437487653220107
diversity: 0.38
 
Results
npmi: 0.0064257546686909885
diversity: 0.405
 
Results
npmi: 0.011036886533741021
diversity: 0.37666666666666665
 
Results
npmi: 0.01675117935273563
diversity: 0.3575
 
Results
npmi: 0.015734180346554376
diversity: 0.342
 


In [10]:
#TOPIC CREATION:
dataset, custom = DATA, True
params = {"num_topics": 15, "random_state": 42}#[(i+1)*10 for i in range(5)], "random_state": random_state}

trainer = Trainer(dataset=dataset,
                  model_name="NMF",
                  params=params,
                  custom_dataset=custom,
                  verbose=True)
results = trainer.train(save=f"results/trump/topics/TR_nmf")

Results
npmi: 0.004451174752642762
diversity: 0.38
 


# CTM

In [7]:
import nltk

#nltk.download("stopwords")
from nltk.corpus import stopwords

In [ ]:
# TC & TD
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
#!pip install contextualized_topic_models
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
for i in range(1):
    dataset, custom = DATA, True
    params = {
        "n_components": [(i+1)*10 for i in range(5)],
        "contextual_size":768
    }

    trainer = Trainer(dataset=dataset,
                      model_name="CTM_CUSTOM",
                      params=params,
                      custom_dataset=custom,
                      verbose=True)
    results = trainer.train(save=f"results/trump/scores/ctm_{i+1}")

Batches:   0%|          | 0/222 [00:00<?, ?it/s]

Epoch: [100/100]	 Seen Samples: [4422400/4425200]	Train Loss: 47.67496832200311	Time: 0:00:09.658039: : 100it [16:20,  9.80s/it]
100%|██████████| 692/692 [00:08<00:00, 84.62it/s] 


Results
npmi: 0.00824862720803795
diversity: 0.99
 


Epoch: [100/100]	 Seen Samples: [4422400/4425200]	Train Loss: 50.25803543068738	Time: 0:00:09.565081: : 100it [16:22,  9.83s/it]
100%|██████████| 692/692 [00:07<00:00, 87.56it/s] 


Results
npmi: 0.006199995908221468
diversity: 0.96
 


Epoch: [68/100]	 Seen Samples: [3007232/4425200]	Train Loss: 53.692074385117174	Time: 0:00:09.632899: : 68it [11:13,  9.65s/it]

In [ ]:
# TOPIC CREATION
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
#!pip install contextualized_topic_models
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
#for i in range(3):
dataset, custom = DATA, True
params = {
    "n_components": 15,#[(i+1)*10 for i in range(5)],
    "contextual_size":768
}

trainer = Trainer(dataset=dataset,
                  model_name="CTM_CUSTOM",
                  params=params,
                  custom_dataset=custom,
                  verbose=True)
results = trainer.train(save=f"results/trump/topics/TR_ctm")

# Dynamic TM

In [14]:
#SETUP
dDATA = "trump_dtm"

In [15]:
dataset = Dataset()
dataset.load_custom_dataset_from_folder(dDATA)

In [16]:
# Prepare data
from data_trump import DataLoader
dataset, custom = dDATA, True
data_loader = DataLoader(dataset)
_, timestamps = data_loader.load_docs()
data = data_loader.load_octis(custom)
data = [" ".join(words) for words in data.get_corpus()]

embedding_model = SentenceTransformer(EM2) #EM3
embeddings = embedding_model.encode(docs, show_progress_bar=True)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/pspaargaren/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
No sentence-transformers model found with name emanjavacas/MacBERTh. Creating a new one with MEAN pooling.


Batches:   0%|          | 0/1383 [00:00<?, ?it/s]

In [17]:
# Match indices
import os
os.listdir(f"./{dataset}")
with open(f"./{dataset}/indexes.txt") as f:
    indices = f.readlines()
    
indices = [int(index.split("\n")[0]) for index in indices]
timestamps = [timestamp for index, timestamp in enumerate(timestamps) if index in indices]
len(data), len(timestamps)

(44252, 44252)

In [18]:
from evaluation import Trainer

params = {
        "nr_topics": 10, #[(i+1)*10 for i in range(5)],
        "min_topic_size": 5,
        "verbose": True,
    }

trainer = Trainer(dataset=dataset,
                      model_name="BERTopic",
                      params=params,
                      bt_embeddings=embeddings,
                      custom_dataset=custom,
                      bt_timestamps=timestamps,
                      topk=5,
                      bt_nr_bins=5,
                      verbose=True)
results = trainer.train(f"DynamicBERTopic_EN")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pspaargaren/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2024-06-10 23:01:33,786 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-10 23:01:58,810 - BERTopic - Dimensionality - Completed ✓
2024-06-10 23:01:58,812 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-10 23:02:02,127 - BERTopic - Cluster - Completed ✓
2024-06-10 23:02:02,128 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-10 23:02:03,773 - BERTopic - Representation - Completed ✓
2024-06-10 23:02:03,774 - BERTopic - Topic reduction - Reducing number of topics
2024-06-10 23:02:04,650 - BERTopic - Topic reduction - Reduced number of topics from 947 to 10
0it [00:00, ?it/s]


KeyError: 1241094607596999936

# MODERN

# MODERN

In [8]:
#SETUP
dDATA = "EN_dtm"

In [9]:
dataset = Dataset()
dataset.load_custom_dataset_from_folder(dDATA)

In [10]:
# Prepare data
from data_flower import DataLoader
dataset, custom = dDATA, True
data_loader = DataLoader(dataset)
_, timestamps = data_loader.load_docs()
data = data_loader.load_octis(custom)
data = [" ".join(words) for words in data.get_corpus()]

embedding_model = SentenceTransformer(EM2) #EM3
embeddings = embedding_model.encode(docs, show_progress_bar=True)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/pspaargaren/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Batches:   0%|          | 0/1445 [00:00<?, ?it/s]

In [11]:
# Match indices
import os
os.listdir(f"./{dataset}")
with open(f"./{dataset}/indexes.txt") as f:
    indices = f.readlines()
    
indices = [int(index.split("\n")[0]) for index in indices]
timestamps = [timestamp for index, timestamp in enumerate(timestamps) if index in indices]
len(data), len(timestamps)

(46236, 46236)

In [12]:
from evaluation import Trainer
#for i in range(3):
params = {
        "nr_topics": [(i+1)*10 for i in range(5)],
        "min_topic_size": 5,
        "verbose": True #,
        # "evolution_tuning": True,
        # "global_tuning": False
    }

trainer = Trainer(dataset=dataset,
                      model_name="BERTopic",
                      params=params,
                      bt_embeddings=embeddings,
                      custom_dataset=custom,
                      bt_timestamps=timestamps,
                      topk=5,
                      bt_nr_bins=5,
                      verbose=True)
results = trainer.train(f"D_EN_NO_M")

2024-06-04 08:18:57,095 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-04 08:19:46,755 - BERTopic - Dimensionality - Completed ✓
2024-06-04 08:19:46,758 - BERTopic - Cluster - Start clustering the reduced embeddings
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been us

marjo
baiga
embellifli


2024-06-04 08:24:06,663 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-04 08:24:33,236 - BERTopic - Dimensionality - Completed ✓
2024-06-04 08:24:33,239 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-04 08:24:36,462 - BERTopic - Cluster - Completed ✓
2024-06-04 08:24:36,463 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-04 08:24:43,775 - BERTopic - Representation - Completed ✓
2024-06-04 08:24:43,778 - BERTopic - Topic reduction - Reducing number of topics
2024-06-04 08:24:49,735 - BERTopic - Topic reduction - Reduced number of topics from 1345 to 20
5it [00:05,  1.08s/it]


regula
baiga
longitudi
embellifli


2024-06-04 08:28:54,694 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-04 08:29:20,922 - BERTopic - Dimensionality - Completed ✓
2024-06-04 08:29:20,925 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-04 08:29:24,190 - BERTopic - Cluster - Completed ✓
2024-06-04 08:29:24,191 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-04 08:29:31,548 - BERTopic - Representation - Completed ✓
2024-06-04 08:29:31,551 - BERTopic - Topic reduction - Reducing number of topics
2024-06-04 08:29:37,694 - BERTopic - Topic reduction - Reduced number of topics from 1349 to 30
5it [00:06,  1.31s/it]


regula
marjo
baiga
stepe


2024-06-04 08:34:22,503 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-04 08:34:58,959 - BERTopic - Dimensionality - Completed ✓
2024-06-04 08:34:58,963 - BERTopic - Cluster - Start clustering the reduced embeddings
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been us

regula
baiga
stepe
marjo
thof
encyclopdia


2024-06-04 08:40:18,315 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-04 08:40:46,224 - BERTopic - Dimensionality - Completed ✓
2024-06-04 08:40:46,227 - BERTopic - Cluster - Start clustering the reduced embeddings
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been us

regula
marjo
sepul
baiga
stepe


In [13]:
print("Het is klaar")

Het is klaar
